In [1]:
%load_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from glob import glob
from shapely.geometry import LineString, Point
import folium
import matplotlib.pyplot as plt
import pytz
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)
import SDS_slope

In [2]:
transects = gpd.read_file("transects_extended.geojson").set_index("id")
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,geometry
id,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nzd0562-0017,nzd0562,NaN,NaN,NaN,0.055,0.0541,0.0608,0.006797,582.0,407.0,0.000019,8.181382,127.722282,11.301428,212.322965,"LINESTRING (174.9093 -36.86951, 174.91331 -36...."
nzd0562-0018,nzd0562,NaN,NaN,NaN,0.045,0.0434,0.0461,0.284894,582.0,372.0,0.022977,9.773159,187.352460,13.687675,216.532589,"LINESTRING (174.90891 -36.86889, 174.91293 -36..."
nzd0563-0000,nzd0563,NaN,NaN,NaN,0.200,0.1756,0.2000,-0.286535,770.0,684.0,0.020982,12.984757,214.309394,14.639310,182.918496,"LINESTRING (176.9374 -39.3345, 176.938 -39.33807)"


In [3]:
new_transects = transects[transects.index.str.startswith("nzd") & (transects.index > "nzd0562") & transects.beach_slope.isna()]
new_transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,geometry
id,,,,,,,,,,,,,,,,


In [4]:
if len(new_transects):
  for site_id in tqdm(new_transects.site_id.unique()):
    df = pd.read_csv(f"data/{site_id}/transect_time_series.csv")
    df.index = pd.to_datetime(df.dates)
    df.drop(columns=["dates", "satname"], inplace=True)
    tides = pd.read_csv("data/{site_id}/tides.csv")
    tides.dates = pd.to_datetime(tides.dates)
    tides.set_index("dates", inplace=True)
    assert all(pd.to_datetime(df.index).round("10min") == tides.index)
    # slope estimation settings
    days_in_year = 365.2425
    seconds_in_day = 24*3600
    settings_slope = {'slope_min':        0.01,                  # minimum slope to trial
                      'slope_max':        0.2,                    # maximum slope to trial
                      'delta_slope':      0.005,                  # slope increment
                      'date_range':       [1999,2020],            # range of dates over which to perform the analysis
                      'n_days':           8,                      # sampling period [days]
                      'n0':               50,                     # parameter for Nyquist criterium in Lomb-Scargle transforms
                      'freqs_cutoff':     1./(seconds_in_day*30), # 1 month frequency
                      'delta_f':          100*1e-10,              # deltaf for identifying peak tidal frequency band
                      'prc_conf':         0.05,                   # percentage above minimum to define confidence bands in energy curve
                      }
    settings_slope['date_range'] = [pytz.utc.localize(datetime(settings_slope['date_range'][0],5,1)),
                                    pytz.utc.localize(datetime(settings_slope['date_range'][1],1,1))]
    beach_slopes = SDS_slope.range_slopes(settings_slope['slope_min'], settings_slope['slope_max'], settings_slope['delta_slope'])

    t = np.array([_.timestamp() for _ in df.index]).astype('float64')
    delta_t = np.diff(t)
    fig, ax = plt.subplots(1,1,figsize=(12,3), tight_layout=True)
    ax.grid(which='major', linestyle=':', color='0.5')
    bins = np.arange(np.min(delta_t)/seconds_in_day, np.max(delta_t)/seconds_in_day+1,1)-0.5
    ax.hist(delta_t/seconds_in_day, bins=bins, ec='k', width=1);
    ax.set(xlabel='timestep [days]', ylabel='counts',
          xticks=7*np.arange(0,20),
          xlim=[0,50], title='Timestep distribution');

    # find tidal peak frequency (can choose 7 or 8 in this case)
    settings_slope['n_days'] = 7
    settings_slope['freqs_max'] = SDS_slope.find_tide_peak(df.index,tides.tide,settings_slope)
    # estimate beach-face slopes along the transects
    slope_est, cis = dict([]), dict([])
    for key in df.keys():
        # remove NaNs
        idx_nan = np.isnan(df[key])
        dates = [df.index[_] for _ in np.where(~idx_nan)[0]]
        tide = tides.tide.to_numpy()[~idx_nan]
        composite = df[key][~idx_nan]
        # apply tidal correction
        tsall = SDS_slope.tide_correct(composite,tide,beach_slopes)
        title = 'Transect %s'%key
        SDS_slope.plot_spectrum_all(dates,composite,tsall,settings_slope, title)
        slope_est[key],cis[key] = SDS_slope.integrate_power_spectrum(dates,tsall,settings_slope)
        print('Beach slope at transect %s: %.3f'%(key, slope_est[key]))
    transects.beach_slope.update(slope_est)
    transects.cil.update({k: v[0] for k,v in cis.items()})
    transects.ciu.update({k: v[1] for k,v in cis.items()})
    transects[transects.index.isin(slope_est.keys())]
  transects.to_file("transects_extended.geojson")